<a href="https://colab.research.google.com/github/vipulgote1999/ML-Data-Scientist-Assignment_GIBots_Pune/blob/main/HyperOpt_GIBots_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq tensorflow_addons numba

In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbMetricsLogger


In [4]:
wandb.login()

wandb: Currently logged in as: vipulgote4 (existence). Use `wandb login --relogin` to force relogin


True

In [16]:
import pandas as pd
import numpy as np
import os, time,json,sys,random,warnings
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler
import keras


import xgboost as xgb
import wandb
from wandb.xgboost import WandbCallback
from sklearn.multioutput import MultiOutputClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import train_test_split

In [6]:
warnings.simplefilter("ignore")
random.seed(42)
np.random.seed(42)
pd.set_option('display.max_columns', None)

In [7]:
!cp "/content/drive/MyDrive/Assignment data/GI Bots/DATA  Scientist Assignment.zip" /content/

In [8]:
!unzip "/content/DATA  Scientist Assignment.zip"

Archive:  /content/DATA  Scientist Assignment.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
train =pd.read_csv("/content/train.csv")
train_labels = pd.read_csv("/content/trainLabels.csv")
train_labels.drop("id", axis=1,inplace=True)
test_data = pd.read_csv("/content/test.csv")
test_data.columns = train.columns

In [10]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  #'mean', 'median', or 'most_frequent'
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bool_transformer = Pipeline(steps=[
#     ('imputer',KNNImputer())
# ])

# here we are preprocessing  numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ], remainder='passthrough',n_jobs=-1)


In [11]:
X = preprocessor.fit_transform(train)
y = train_labels[:9999]

In [12]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:


run = wandb.init(project = "GiBots-xgboost-finetune")

clf = MultiOutputClassifier(xgb.XGBClassifier(n_estimators=200,device="cuda", random_state=42,
                      objective='binary:logistic'))

clf.fit(X_train,y_train, callbacks=[WandbCallback(log_model=True)])
print(f"model score: {clf.score(X_val,y_val)}")

from sklearn.metrics import f1_score

preds = clf.predict(X_val)
print(f"f1_score: {f1_score(y_val, preds, average='micro')}")

model score: 0.7795
f1_score: 0.847821020741086


In [23]:
X_train.shape,y_train.shape

((7999, 34855), (7999, 33))

In [27]:

# X_train, X_val, y_train, X_val = train_test_split(X, y.values, test_size=0.2, random_state=42)
def objective(params):
#     test_model = xgb.XGBClassifier(objective='binary:logistic',random_state=42,verbose=1,early_stopping_rounds=5,verbose_eval=1,**params)
    test_model = MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic',random_state=42,device="cuda",
                                   verbose=1,**params))
#     test_model.fit(X_train, y_train,eval_set=[(X_val, y_val)])
    test_model.fit(X_train, y_train,callbacks=[WandbCallback(log_model=True)])

    #y_pred = test_model.predict(X_test)
    #print(y_pred.shape,y_test.shape)
    #regressor.score(X_test, y_test)
    neg_metric = -test_model.score(X_val, y_val)  # r2-score

    return {'loss': neg_metric, 'status': STATUS_OK}

In [28]:
search_space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
      'n_estimators': hp.choice('n_estimators', range(50, 530,30)),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
      'reg_lambda': hp.uniform('reg_lambda', 0, 1)
}

# stopping becuase taking log time

In [29]:
%%time
trials = Trials()
best_params = fmin(objective, search_space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters
print(f"Best hyperparameters: {best_params}")

# Use the best hyperparameters to train the final model
model = MultiOutputClassifier(xgb.XGBClassifier(**best_params))

  3%|▎         | 3/100 [12:04<6:30:41, 241.66s/trial, best loss: -0.79]


KeyboardInterrupt: 

# View Training Metrics at this link: https://wandb.ai/existence/GiBots-xgboost-finetune

# Let do HyperParameter Search by weights and bias Sweeps method

In [22]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
            "values": [5, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.2, 0.05]
        },
        "subsample": {
            "values": [1, 0.5, 0.7]
        },
        'n_estimators':{
            "values":[100,200,250,300,350,400,500]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="Bayesian-XGBoost-sweeps")

400 response executing GraphQL.
{"errors":[{"message":"Invalid sweep config: invalid hyperparameter configuration: n_estimators","path":["upsertSweep"]}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: Invalid sweep config: invalid hyperparameter configuration: n_estimators (<Response [400]>)


UsageError: Invalid sweep config: invalid hyperparameter configuration: n_estimators

In [18]:
def train():
  wandb.init()
  config = wandb.config
  model = MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic',random_state=42,device="cuda",verbose=1,
                                                  booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample,n_estimators=config.n_estimators))

  model.fit(X_train, y_train,callbacks=[WandbCallback(log_model=True)])

  wandb.log({"accuracy": model.score(X_val, y_val)})

In [19]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: tj04aihg with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 12
wandb: 	n_estimators: 300
wandb: 	subsample: 1


accuracy,▁
epoch,▄▇▆▅▄▃▂█▇▆▅▄▃▁█▇▆▄▃▂▅▄▃▂▁▇▆▅▄▃▂▁▇▆▅▄▃▂█▇
accuracy,0.7885
epoch,299


wandb: Agent Starting Run: 7puovxtm with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 3
wandb: 	n_estimators: 50
wandb: 	subsample: 0.3


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,49


Run 7puovxtm errored: XGBoostError('[18:40:23] /workspace/src/gbm/gblinear.cc:247: Check failed: importance_type == "weight" (gain vs. weight) : gblinear only has `weight` defined for feature importance.\nStack trace:\n  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x447a4a) [0x7fccdf869a4a]\n  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x44d43f) [0x7fccdf86f43f]\n  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4a6a24) [0x7fccdf8c8a24]\n  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGBoosterFeatureScore+0x6c7) [0x7fccdf597a07]\n  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7fcdabe9de2e]\n  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7fcdabe9a493]\n  [bt] (6) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa3e9) [0x7fcdabec33e9]\n  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0

...

wandb: Agent Starting Run: 2aw2flk6 with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 6
wandb: 	n_estimators: 100
wandb: 	subsample: 0.3


accuracy,▁
epoch,▃█▇▅▅▄▂▁▆▆▅▃▂▂▇▇▄▄▃██▅▅▅▂▂▁▆▆▃▃▂▇▇▄▄▄▁██
accuracy,0.7355
epoch,99


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2s8c8aj1 with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 9
wandb: 	n_estimators: 200
wandb: 	subsample: 0.5


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,199


Run 2s8c8aj1 errored: XGBoostError('[18:59:34] /workspace/src/gbm/gblinear.cc:247: Check failed: importance_type == "weight" (gain vs. weight) : gblinear only has `weight` defined for feature importance.\nStack trace:\n  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x447a4a) [0x7fccdf869a4a]\n  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x44d43f) [0x7fccdf86f43f]\n  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4a6a24) [0x7fccdf8c8a24]\n  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGBoosterFeatureScore+0x6c7) [0x7fccdf597a07]\n  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7fcdabe9de2e]\n  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7fcdabe9a493]\n  [bt] (6) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa3e9) [0x7fcdabec33e9]\n  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0

# See above hyperprameter results here: https://wandb.ai/existence/XGBoost-sweeps/sweeps/4mlcg3hs

In [ ]:
# 2nd time model optimization


In [23]:
sweep_config = {
    "method": "bayes", # try grid or random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
            "values": [5, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.2, 0.05]
        },
        "subsample": {
            "values": [1, 0.5, 0.7]
        },
        'n_estimators':{
            "values":[100,200,250,300,350,400,500]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="Bayesian-XGBoost-sweeps")

Create sweep with ID: zc4r45wr
Sweep URL: https://wandb.ai/existence/Bayesian-XGBoost-sweeps/sweeps/zc4r45wr


In [24]:
def train():
  wandb.init()
  config = wandb.config
  model = MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic',random_state=42,device="cuda",verbose=1,
                                                  booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample,n_estimators=config.n_estimators))

  model.fit(X_train, y_train,callbacks=[WandbCallback(log_model=True)])

  wandb.log({"accuracy": model.score(X_val, y_val)})

# see bayes method results here: https://wandb.ai/existence/Bayesian-XGBoost-sweeps/sweeps/zc4r45wr

In [ ]:
wandb.agent(sweep_id, train, count=100)

wandb: Agent Starting Run: 0kobctux with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 5
wandb: 	n_estimators: 200
wandb: 	subsample: 1


accuracy,▁
epoch,▃█▇▅▅▄▂▁▆▆▅▃▂▂▇▇▄▄▃██▅▅▅▂▂▁▆▆▃▃▂▇▇▄▄▄▁██
accuracy,0.762
epoch,199


wandb: Agent Starting Run: 396vi4tm with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 5
wandb: 	n_estimators: 500
wandb: 	subsample: 1
